In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from autobook.libgen import Libgen

data = Libgen(q="cloud cuckoo land")
df = data.search()
df = df.filter("Anthony Doerr")


In [2]:
df.get_filtered_results()

,author(s),series,title,language,file,mirror1,mirror2,md5,topic,extension,size
2,Anthony Doerr,,Cloud Cuckoo Land,English,EPUB / 4.98 Mb,http://library.lol/fiction/7F2FCC965C0414C4C3F...,https://library.bz/fiction/edit/7F2FCC965C0414...,7F2FCC965C0414C4C3F105FC56697F64,fiction,epub,4.98 Mb


In [7]:
df.download_file()

FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\jjfan\\PYTHON-PKG_WD\\autobook\\books\\7F2FCC965C0414C4C3F105FC56697F64'